In [ ]:
s =time.time()
adder = False
count=0
for lis in dat['collection']:
    lets = lis.split("\n")
    counter=0
    for let in lets:
        res = re.findall(r'\w+', let)
        graph = res[-4]
        if graph =="histogram":
            starting = time.time()
            xlabel = res[3]
            ylabel = res[6]
            x = df[xlabel]
            x = cupy.fromDlpack(x.to_dlpack())
            frequencies, edges = cupy.histogram(x, bins=10)
            curve=hv.Histogram((edges.get(), frequencies.get()))
            if not adder: adder = curve
            else: adder+=curve
            print("time in histogram :", time.time() -starting)
        elif graph =="bar":
            starting = time.time()
            ylabel = res[5]
            x=df.groupby(ylabel).count()
            x.reset_index(inplace=True) 
            lis = list(zip(x[ylabel].values_host, x.iloc[:, 1].values_host))
            curve = hv.Bars(lis).opts(invert_axes=True)
            if not adder: adder = curve
            else: adder+=curve
            print("time in inverted bar :", time.time() -starting)
        if graph =="line":
            starting = time.time()
            xlabel = res[2]
            factor = re.search(xlabel + '(.*)y', "".join(res)).group(1)
            ylabel = "Record"
            x = cudf.DatetimeIndex(df[xlabel])
            if factor == 'dayofweek':
                x = cudf.DataFrame(x.dayofweek, columns = ["date"])
            elif factor =='month':
                x = cudf.DataFrame(x.month, columns = ["date"])
            elif factor =="year":
                x = cudf.DataFrame(x.year, columns = ["date"])
            else:
                x = cudf.DataFrame(x, columns = ["date"])
            x["count"] = x['date']
            x = x.groupby("date").count()
            x.reset_index(inplace=True)
            x = x.sort_values('date')
            lis = list(zip(x['date'].values_host, x['count'].values_host))
            curve = hv.Curve(lis, hv.Dimension(xlabel), "Number of " +ylabel)
            if not adder: adder = curve
            else: adder+=curve
            print("time in linecurve :", time.time() -starting)
        elif graph == "scatter":
            starting = time.time()
            xlabel = res[2]
            ylabel = res[4]
            x = df[xlabel]
            y = df[ylabel]
            z = cudf.concat([x,y],axis=1)
            curve = datashade(hv.Scatter(z))
            count+=1
            #For heatmaps, place last cells code here
            if not adder: adder = curve
            else: adder+=curve
            print("time in scatterplot :", time.time() -starting)

print("total time : ", time.time()-s)